In [1]:
# !pip install -U spacy
# Use the command to download the SpaCy English model:
# python -m spacy download en_core_web_sm
import spacy
# Import SpaCy English model
import en_core_web_sm
from spacy import displacy

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import nltk
import re, pprint, string, os, math
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import bs4 as bs
import urllib.request
import heapq
from itertools import islice
from nltk import ngrams
from collections import Counter
from pprint import pprint
import heapq
from collections import defaultdict
from string import punctuation
from heapq import nlargest

In [3]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = en_core_web_sm.load()

In [4]:
# 1.1 (0.3 points) Copy the code examples to scrape the webpage in BeautifulSoup
# Text Data Location: https://www.yahoo.com/news/suspected-parade-gunman-traveled-wisconsin-033707038.html
def scrape_webpage(url):
    ''' Use BeautifulSoup to scrape the webpage text contents'''
    scraped_textdata = urllib.request.urlopen(url)
    textdata = scraped_textdata.read()
    parsed_textdata = bs.BeautifulSoup(textdata, 'lxml')
    paragraphs = parsed_textdata.find_all('p')
    formated_text = ""
    
    for para in paragraphs:
        formated_text += para.text
    
    return formated_text

webtext = scrape_webpage('https://techcrunch.com/2022/07/07/elon-musk-twitter-deal-wapo-report/')
mytext = nlp(webtext)

In [5]:
print(mytext)

According to a new report from The Washington Post that relies heavily on anonymous sources, the world’s richest man is still looking for a way out of a $44 billion deal of his own making.The Post reports that we’ll probably be seeing a “change in direction from Musk’s team” soon, according to its unnamed sources familiar with his plans. Further, Musk and friends have reportedly “stopped engaging in certain discussions” pertaining to funding the deal. Oracle co-founder Larry Ellison, Sequoia Capital, Andreessen Horowitz, Binance and Fidelity are all on board for financing after signing on in early May.Musk appears to again be sowing doubt about Twitter’s bot estimates with these new shenanigans, but how and if those supposed concerns would even impact a deal he’s already signed onto is far from clear. The terms of the merger agreement include a stipulation that Musk would have to cough up $1 billion if he backs out of the deal, though it certainly looks like he’s combing through the fi

In [6]:
# 1.2 (4.7 points) Write the code for NER in SpaCy
# 1.2.1 (0.2 points) Count all the named entities in the document
# Count every named entity
labels = [x.label_ for x in mytext.ents]
from collections import Counter
Counter(labels)

Counter({'ORG': 7,
         'MONEY': 4,
         'PERSON': 17,
         'GPE': 1,
         'DATE': 6,
         'PRODUCT': 1,
         'PERCENT': 1,
         'WORK_OF_ART': 1,
         'ORDINAL': 1,
         'CARDINAL': 1})

In [7]:
# 1.2.2 (0.5 points) Count the most frequent tokens for the entire document
# Get most frequent tokens
items = [x.text for x in mytext.ents]
Counter(items).most_common(5)

[('Musk', 12),
 ('Twitter', 4),
 ('$44 billion', 2),
 ('The Washington Post', 1),
 ('Post', 1)]

In [8]:
# 1.2.3 (2.0 points) Pick a random integer K using Python random module, then pick
# three consecutive sentences starting with Kth, and print these sentences. 
# Note that you must make sure all picked sentences are in the document.
print('''
Number of words\t\t{num_words}
Number of sentences\t{num_sent}
'''.format(
    num_words=len(mytext),  # grab number of tokens
    num_sent=len(list(mytext.sents)),
))


import random
# Method 1: Use Regex to split sentences then print 3 consecutive sentences
K = random.randint(0,18-3) 
print('The random K number is: ', K)

import re
' '.join(re.split(r'(?<=[.:;])\s', webtext)[K:K+3])



Number of words		556
Number of sentences	18

The random K number is:  0


'According to a new report from The Washington Post that relies heavily on anonymous sources, the world’s richest man is still looking for a way out of a $44 billion deal of his own making.The Post reports that we’ll probably be seeing a “change in direction from Musk’s team” soon, according to its unnamed sources familiar with his plans. Further, Musk and friends have reportedly “stopped engaging in certain discussions” pertaining to funding the deal. Oracle co-founder Larry Ellison, Sequoia Capital, Andreessen Horowitz, Binance and Fidelity are all on board for financing after signing on in early May.Musk appears to again be sowing doubt about Twitter’s bot estimates with these new shenanigans, but how and if those supposed concerns would even impact a deal he’s already signed onto is far from clear.'

In [9]:
#Method 2:  Convert the text to a list of sentences then print 3 consecutive items in the list.
sentences = [sent.text.strip() for sent in mytext.sents]
print('Number of sentences: ', len(sentences))
K = random.randint(0,18-3)
print('K number is: ', K)
i = 0
while i < len(sentences):
    if i == K:
        cons_sents = sentences[i] + ' ' + sentences[i+1]+ ' ' + sentences[i+2]
        print("3 CONSECUTIVE SENTENCES AT Kth: \n", cons_sents)
        #print("3 CONSECUTIVE SENTENCES AT Kth: \n", sentences[i], sentences[i+1], sentences[i+2])
        break
    i +=1



Number of sentences:  18
K number is:  10
3 CONSECUTIVE SENTENCES AT Kth: 
 The whole thing seems like a thin charade for a guy hoping to weasel out of a badly timed acquisition. But unfortunately for Musk and the Twitter employees who’ve been on this multimonth acquisition roller coaster, the ink is already dry on the deal. Last month, Musk participated in a companywide call with Twitter employees — a rare instance of actually engaging with the entity he’s on the hook to buy and a gesture that signaled he was moving forward with the deal he has to move forward with.


In [10]:
# 1.2.4 (0.5 points) Extract part-of-speech and lemmatize these consecutive sentences
cons_sents1 = nlp(cons_sents)
for token in cons_sents1:
    print("Word: '" + token.text +"'" , " POS: " + token.pos_, " Lemma: " + token.lemma_)

Word: 'The'  POS: DET  Lemma: the
Word: 'whole'  POS: ADJ  Lemma: whole
Word: 'thing'  POS: NOUN  Lemma: thing
Word: 'seems'  POS: VERB  Lemma: seem
Word: 'like'  POS: ADP  Lemma: like
Word: 'a'  POS: DET  Lemma: a
Word: 'thin'  POS: ADJ  Lemma: thin
Word: 'charade'  POS: NOUN  Lemma: charade
Word: 'for'  POS: ADP  Lemma: for
Word: 'a'  POS: DET  Lemma: a
Word: 'guy'  POS: NOUN  Lemma: guy
Word: 'hoping'  POS: VERB  Lemma: hope
Word: 'to'  POS: PART  Lemma: to
Word: 'weasel'  POS: VERB  Lemma: weasel
Word: 'out'  POS: ADP  Lemma: out
Word: 'of'  POS: ADP  Lemma: of
Word: 'a'  POS: DET  Lemma: a
Word: 'badly'  POS: ADV  Lemma: badly
Word: 'timed'  POS: VERB  Lemma: time
Word: 'acquisition'  POS: NOUN  Lemma: acquisition
Word: '.'  POS: PUNCT  Lemma: .
Word: 'But'  POS: CCONJ  Lemma: but
Word: 'unfortunately'  POS: ADV  Lemma: unfortunately
Word: 'for'  POS: ADP  Lemma: for
Word: 'Musk'  POS: PROPN  Lemma: Musk
Word: 'and'  POS: CCONJ  Lemma: and
Word: 'the'  POS: DET  Lemma: the
Word: '

In [11]:
# 1.2.5 (0.5 points) Get and print the entity annotation for each token of the Kth sentence

# Get the token-level entity annotations using the BILUO tagging scheme to describe the entity boundaries
pprint([(X, X.ent_iob_, X.ent_type_) for X in cons_sents1])


[(The, 'O', ''),
 (whole, 'O', ''),
 (thing, 'O', ''),
 (seems, 'O', ''),
 (like, 'O', ''),
 (a, 'O', ''),
 (thin, 'O', ''),
 (charade, 'O', ''),
 (for, 'O', ''),
 (a, 'O', ''),
 (guy, 'O', ''),
 (hoping, 'O', ''),
 (to, 'O', ''),
 (weasel, 'O', ''),
 (out, 'O', ''),
 (of, 'O', ''),
 (a, 'O', ''),
 (badly, 'O', ''),
 (timed, 'O', ''),
 (acquisition, 'O', ''),
 (., 'O', ''),
 (But, 'O', ''),
 (unfortunately, 'O', ''),
 (for, 'O', ''),
 (Musk, 'B', 'PERSON'),
 (and, 'O', ''),
 (the, 'O', ''),
 (Twitter, 'B', 'PERSON'),
 (employees, 'O', ''),
 (who, 'O', ''),
 (’ve, 'O', ''),
 (been, 'O', ''),
 (on, 'O', ''),
 (this, 'O', ''),
 (multimonth, 'B', 'ORDINAL'),
 (acquisition, 'O', ''),
 (roller, 'O', ''),
 (coaster, 'O', ''),
 (,, 'O', ''),
 (the, 'O', ''),
 (ink, 'O', ''),
 (is, 'O', ''),
 (already, 'O', ''),
 (dry, 'O', ''),
 (on, 'O', ''),
 (the, 'O', ''),
 (deal, 'O', ''),
 (., 'O', ''),
 (Last, 'B', 'DATE'),
 (month, 'I', 'DATE'),
 (,, 'O', ''),
 (Musk, 'B', 'PERSON'),
 (participated, 'O

In [12]:
# Get the named entities
pprint([(X.text, X.label_) for X in cons_sents1.ents])

[('Musk', 'PERSON'),
 ('Twitter', 'PERSON'),
 ('multimonth', 'ORDINAL'),
 ('Last month', 'DATE'),
 ('Musk', 'PERSON')]


In [13]:
# 1.2.6 (0.5 points) Visualize the entities and dependencies of Kth sentence

displacy.render(cons_sents1, jupyter=True, style='ent')

displacy.render(cons_sents1, style='dep', jupyter = True, options = {'distance': 80})

In [14]:
cons_sents1.user_data["title"] = "VISUALIZATION OF THE ENTITIES OF 3 CONSECUTIVE SENTENCES STARTING AT Kth SENTENCE"
displacy.render(cons_sents1, style="ent")

In [15]:
# 1.2.7 (0.5 points) Visualize all the entities in the document
mytext.user_data["title"] = "VISUALIZATION OF ALL THE ENTITIES IN THE DOCUMENT"
displacy.render(mytext, jupyter=True, style='ent')



In [16]:
displacy.render(mytext, style='dep', jupyter = True, options = {'distance': 80})

In [17]:
print(webtext)

According to a new report from The Washington Post that relies heavily on anonymous sources, the world’s richest man is still looking for a way out of a $44 billion deal of his own making.The Post reports that we’ll probably be seeing a “change in direction from Musk’s team” soon, according to its unnamed sources familiar with his plans. Further, Musk and friends have reportedly “stopped engaging in certain discussions” pertaining to funding the deal. Oracle co-founder Larry Ellison, Sequoia Capital, Andreessen Horowitz, Binance and Fidelity are all on board for financing after signing on in early May.Musk appears to again be sowing doubt about Twitter’s bot estimates with these new shenanigans, but how and if those supposed concerns would even impact a deal he’s already signed onto is far from clear. The terms of the merger agreement include a stipulation that Musk would have to cough up $1 billion if he backs out of the deal, though it certainly looks like he’s combing through the fi

In [18]:
# 2 (5 points) De-Identification:
#  2.1 De-identify all person names (PERSON) in the webpage document with [REDACTED]
mytext = nlp(webtext)
person_list = [ent.text for ent in mytext.ents if ent.label_ == 'PERSON']
pattern = '|'.join(f'\\b{w}\\b' for w in person_list)
print('This is the regex pattern: \n ', pattern, '\n')
def redacted_persons(webtext):
    return re.sub(pattern, '[REDACTED]', webtext)

print(redacted_persons(webtext))

This is the regex pattern: 
  \bMusk\b|\bMusk\b|\bLarry Ellison\b|\bAndreessen Horowitz\b|\bMusk\b|\bMusk\b|\bMusk\b|\bTwitter bidMusk\b|\bMusk\b|\bMusk\b|\bTwitter\b|\bMusk\b|\bMusk\b|\bMusk\b|\bMusk\b|\bMusk\b|\bElon Musk\b 

According to a new report from The Washington Post that relies heavily on anonymous sources, the world’s richest man is still looking for a way out of a $44 billion deal of his own making.The Post reports that we’ll probably be seeing a “change in direction from [REDACTED]’s team” soon, according to its unnamed sources familiar with his plans. Further, [REDACTED] and friends have reportedly “stopped engaging in certain discussions” pertaining to funding the deal. Oracle co-founder [REDACTED], Sequoia Capital, [REDACTED], Binance and Fidelity are all on board for financing after signing on in early May.[REDACTED] appears to again be sowing doubt about [REDACTED]’s bot estimates with these new shenanigans, but how and if those supposed concerns would even impact a

In [19]:
# and visualize them
redacted = nlp(redacted_persons(webtext))
redacted.user_data["title"] = "VISUALIZATION AFTER REPLACED 'PERSON' WITH [REDACTED]"
displacy.render(redacted, jupyter=True, style='ent')

In [20]:
mytext.user_data["title"] = "VISUALIZATION WITH NAMED ENTITIES BEFORE REPLACE PERSON FOR '[REDACTED]'"
displacy.render(mytext, jupyter=True, style='ent')

### References:
Liao, D. (2022). NLP Code Example for Named Entity Recognition (NER).  George Mason University.

Hatmaker, T. (2022). It sounds like Elon Musk is still trying to get out of his own Twitter deal.  Tech Crunch.  Retrieved on 8 July 2022 from https://techcrunch.com/2022/07/07/elon-musk-twitter-deal-wapo-report/

Stringham, J. (2018). Quick post on spaCy. Retrieved on 6 July 2022, from https://jessicastringham.net/2018/06/22/spacy/

spaCy (2022).  Linguistic Features.  Retrieved on 6 July 2002, from https://spacy.io/usage/linguistic-features#sbd

kederrac (2020). How do I replace words (in a txt file) that match my list of strings? Stackoverflow.com. Retrieved on 8 July 2022 from
https://stackoverflow.com/questions/61178196/how-do-i-replace-words-in-a-txt-file-that-match-my-list-of-strings